In [2]:
import pandas as pd

In [4]:
# create dataframe to store district data
district_data = pd.read_csv('dime_recipients_1979_2024.csv')


# isolates years 2002-2022
after_2002 = district_data['fecyear'] >= 2002
before_2022 = district_data['fecyear'] <= 2022
district_data = district_data[after_2002 & before_2022]




/var/folders/11/1zlp690171z58pnrzbnr1s380000gn/T/ipykernel_4657/113295295.py:2: DtypeWarning: Columns (10,12,15,16,17,41,43,44,45,47,49,50,54,55,58,59,60,61,63) have mixed types. Specify dtype option on import or set low_memory=False.
  district_data = pd.read_csv('dime_recipients_1979_2024.csv')


In [5]:
# isolates House candidates
house_candidates = district_data['seat'] == 'federal:house'
district_data = district_data[house_candidates]

# checks that House candidates were selected
district_data['seat'].unique()


array(['federal:house'], dtype=object)

In [6]:
# isolating statutory candidates for the current cycle
statutory_curr_cycle = district_data['fec.cand.status'] == 'C'

district_data = district_data[statutory_curr_cycle]

# checks that statutory candidates for current cycle were selected
# print(district_data['fec.cand.status'].unique())

district_data


,election,cycle,fecyear,bonica.rid,bonica.cid,name,lname,ffname,fname,mname,...,FEC.ID,NID,before.switch.ICPSR,after.switch.ICPSR,party.orig,nimsp.party,nimsp.candidate.ICO.code,nimsp.district,nimsp.office,nimsp.candidate.status
268,fd2002,2002,2002.0,cand53,1.056417e+08,"conyers, john jr",conyers,john,john,NaN,...,C00409797,N00004029,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
269,fd2004,2004,2004.0,cand53,1.056417e+08,"conyers, john jr",conyers,john,john,NaN,...,C00409797,N00004029,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
270,fd2006,2006,2006.0,cand53,1.056417e+08,"conyers, john jr",conyers,john,john,NaN,...,C00409797,N00004029,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
272,fd2008,2008,2008.0,cand53,1.056417e+08,"conyers, john jr",conyers,john,john,NaN,...,C00409797,N00004029,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
273,fd2010,2010,2010.0,cand53,1.056417e+08,"conyers, john jr",conyers,john,john,NaN,...,C00409797,N00004029,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72450,fd2022,2022,2022.0,cand1082,5.000000e+15,"chu, judy",chu,judy,judy,NaN,...,C00458125,N00030600,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
72451,fd2022,2022,2022.0,cand223228,4.623034e+09,"schiff, adam",schiff,adam,adam,NaN,...,C00343871,N00009585,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
72452,fd2022,2022,2022.0,cand1085,5.302098e+09,"garamendi, john",garamendi,john,john,NaN,...,C00462697,N00030856,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN
73123,fd2022,2022,2022.0,cand1035,5.593459e+09,"grayson, alan mark",grayson,alan mark,alan,mark,...,C00424713,N00028418,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN


In [7]:
#dropping observations with no district

has_district = (district_data['district'].isnull())==False

district_data = district_data[has_district]

#ensures that every candidate has a district
#print(district_data['district'].notnull().unique()) 

district_data['ico.status']

268      I
269      I
270      I
272      I
273      I
        ..
72450    I
72451    I
72452    I
73123    O
77766    C
Name: ico.status, Length: 15863, dtype: object

In [8]:
#dropping observations for Alaska, DC, Hawaii, Guam, American Samoa, Virigin Islands, Puerto Rico
excluded_regions = district_data['state'].isin(['AK','DC','HI','AS','VI','PR'])

included_regions = ~excluded_regions

district_data = district_data[included_regions]

#ensures excluded districts were properly excluded
#print(district_data['state'].isin(['AK','DC','HI','AS','VI','PR']).unique)

In [13]:
# determined amt of candidates based on district, fecyear, state, and incumbency status
grouped_districts = district_data.groupby(['district','fecyear','state','ico.status'])['name'].nunique().reset_index(name='num_candidates')

# finding total incumbents
total_incumbents = grouped_districts[['district','fecyear','num_candidates','ico.status']]
total_incumbents = total_incumbents[ total_incumbents['ico.status'] == 'I']

#represents total_incumbents per district and year
total_incumbents = total_incumbents.groupby(['fecyear','district'])['num_candidates'].sum().reset_index(name='total_incumbents')

# renames 'fecyear' series to 'year' for readability t
grouped_districts = grouped_districts.rename(columns={'fecyear': 'year'})
total_incumbents = total_incumbents.rename(columns={'fecyear': 'year'})

# assigns designation for zero, one, or multiple incumbents

grouped_districts['zero_incumbs']= grouped_districts.apply(lambda x: x['ico.status']!= 'I', axis =1)

grouped_districts['one_incumb']= grouped_districts.apply(lambda x: x['num_candidates'] == 1 and x['ico.status']== 'I', axis =1)

grouped_districts['multiple_incumbs']= grouped_districts.apply(lambda x: x['num_candidates'] >=2 and x['ico.status']== 'I', axis =1)



# copies grouped_districts such that num_candidates can be accessed while correctly displaying state-district-year level data


grouped_districts_copy = pd.DataFrame()
grouped_districts_copy = grouped_districts

# removes num_candidates column
grouped_districts = grouped_districts.drop('num_candidates', axis=1)

# saves dataset
grouped_districts.to_csv('/Users/divya/Documents/ra_data_task/data/cleaned_district_data.csv')

grouped_districts






Index(['fecyear', 'district', 'total_incumbents'], dtype='object')


,district,year,state,ico.status,zero_incumbs,one_incumb,multiple_incumbs
0,AL01,2002.0,AL,O,True,False,False
1,AL01,2004.0,AL,C,True,False,False
2,AL01,2004.0,AL,I,False,True,False
3,AL01,2006.0,AL,C,True,False,False
4,AL01,2008.0,AL,C,True,False,False
...,...,...,...,...,...,...,...
8698,WY01,2018.0,WY,I,False,True,False
8699,WY01,2020.0,WY,C,True,False,False
8700,WY01,2020.0,WY,I,False,True,False
8701,WY01,2022.0,WY,C,True,False,False


In [56]:
# creates data frame with year, num_candidates, and incumbency status 
incumbents_per_year = grouped_districts_copy[['year', 'num_candidates', 'ico.status']]


# finds total incumbents for each year
total_incumbents = total_incumbents.groupby('year')['total_incumbents'].sum().reset_index(name='total_incumbents')

# amount/proportion of districts with zero incumbents per year
zero_incumb = incumbents_per_year.groupby('year').apply(lambda row: ((row['ico.status']!= 'I') ).sum()).reset_index(name='zero_incumb')

# amount/proportion of districts with one incumbent per year
one_incumb = incumbents_per_year.groupby('year').apply(lambda row: ( (row['num_candidates']==1) & (row['ico.status']== 'I') ).sum()).reset_index(name='one_incumb')


# amount/proportion of districts with multiple incumbents per year
multiple_incumb = incumbents_per_year.groupby('year').apply(lambda row: ( (row['num_candidates']>=2) & (row['ico.status']== 'I') ).sum()).reset_index(name='multiple_incumb')




# merge results into one table
incumbents_per_year = zero_incumb.merge(one_incumb, on='year', how='left')
incumbents_per_year = incumbents_per_year.merge(multiple_incumb, on='year', how='left')
incumbents_per_year = incumbents_per_year.merge(total_incumbents, on='year', how='left')

incumbents_per_year['share_zero_incumb'] = incumbents_per_year['zero_incumb'] / total_incumbents['total_incumbents']
incumbents_per_year['share_one_incumb'] = incumbents_per_year['one_incumb'] / total_incumbents['total_incumbents']
incumbents_per_year['share_multiple_incumb'] = incumbents_per_year['multiple_incumb'] / total_incumbents['total_incumbents']

incumbents_per_year.drop('zero_incumb', axis =1, inplace = True)
incumbents_per_year.drop('one_incumb', axis =1, inplace = True)
incumbents_per_year.drop('multiple_incumb', axis =1, inplace = True)
incumbents_per_year.drop('total_incumbents', axis =1, inplace = True)



incumbents_per_year





/var/folders/11/1zlp690171z58pnrzbnr1s380000gn/T/ipykernel_1114/1121419830.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zero_incumb = incumbents_per_year.groupby('year').apply(lambda row: ((row['ico.status']!= 'I') ).sum()).reset_index(name='zero_incumb')
/var/folders/11/1zlp690171z58pnrzbnr1s380000gn/T/ipykernel_1114/1121419830.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  one_incumb = incumb

,year,share_zero_incumb,share_one_incumb,share_multiple_incumb
0,2002.0,0.827586,0.920424,0.039788
1,2004.0,0.918478,0.983696,0.008152
2,2006.0,0.839806,0.890777,0.053398
3,2008.0,0.902778,0.953704,0.023148
4,2010.0,0.931280,0.971564,0.014218
5,2012.0,0.960465,0.867442,0.065116
6,2014.0,0.854762,0.980952,0.009524
7,2016.0,0.902613,1.000000,0.000000
8,2018.0,1.068558,0.981087,0.009456
9,2020.0,1.022936,0.949541,0.025229
